In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))
PATH_FOR_RESULT: Path = NOTEBOOK_PATH.parent.joinpath('RESULT')

# 读取数据

我们已获得的数据，有 csv、xls、xlsx、txt 这几种。我们分别处理。

另外，前面在[处理压缩文件](处理压缩文件.ipynb)章节，我们忽略了各交易所的数据并不一直。

因此我们不能下载好所有数据一次性解压缩、一次性读取。我们只能逐交易所地进行。

这也意味着，同样的读取 csv 文件，中金所和大商所的就有所不同。

## 中金所

中金所的数据相对比较简单，一来是中金所品种比较少，三个国债期货、三个股指期货再加一个股指期权；二来是中金所的数据比较“干净”，不像前面处理大商所那么头疼。

In [2]:
from typing import Dict, List, Any
import datetime as dt
import csv

def read_cffex_data(data_file: Path) -> List[Dict[str, Any]]:
    """
    读取中国金融期货交易所（中金所，CFFEX）的历史交易数据 (csv 文件)。
    :param data_file: Path，待读取的文件。
    :return: list，每一个项都是一个以 str 为 key 的字典。
    """
    result: List[Dict[str, Any]] = []
    
    # 从文件名中获得日期。
    filename: str = data_file.name[:8]
    date: dt.date = dt.date(
        year=int(filename[:4]),
        month=int(filename[4:6]),
        day=int(filename[6:8])
    )
    
    # 打开 <data_file> 读取数据。
    with open(data_file, mode='r', encoding='gbk') as csv_file:
        reader = csv.DictReader(csv_file)
        
        # 按行循环读取。
        for row in reader:
            
            # 忽略 <合约代码> 列为 <小计>、<合计> 的行
            if row['合约代码'] == '小计' or row['合约代码'] == '合计' or '-' in row['合约代码']:
                continue
            try:
                result.append(
                    {
                        'exchange': 'CFFEX',
                        'date': date,
                        'symbol': row['合约代码'].strip(),
                        'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                        'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                        'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                        'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                        'settlement': float(row['今结算']),
                        'previous_settlement': float(row['前结算']),
                        'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                        'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                        'open_interest': int(float(row['持仓量'])),
                        'change_on_close': float(row['涨跌1']),
                        'change_on_settlement': float(row['涨跌2']),
                        'change_on_open_interest': int(float(row['持仓变化'])),
                    }
                )
            except ValueError as e:
                print(f'读取文件 {csv_file} 时发生错误。发生错误的行内容为：\n\t{row}')
    return result

# 测试
# 用前面第4轮解压的文件，选了一个离现在比较近的日期，内容比较多一些。
test_data_file: Path = PATH_FOR_RESULT.joinpath('temp4', '20210521_1.csv')

for item in read_cffex_data(test_data_file):
    print(item)

{'exchange': 'CFFEX', 'date': datetime.date(2021, 5, 21), 'symbol': 'IC2105', 'open': 6566.0, 'high': 6576.0, 'low': 6505.6, 'close': 6535.2, 'settlement': 6535.64, 'previous_settlement': 6562.6, 'volume': 18203, 'amount': 2380350.86, 'open_interest': 0, 'change_on_close': -27.4, 'change_on_settlement': -26.96, 'change_on_open_interest': -19933}
{'exchange': 'CFFEX', 'date': datetime.date(2021, 5, 21), 'symbol': 'IC2106', 'open': 6539.8, 'high': 6544.8, 'low': 6471.0, 'close': 6495.2, 'settlement': 6497.6, 'previous_settlement': 6526.8, 'volume': 64554, 'amount': 8395711.82, 'open_interest': 125308, 'change_on_close': -31.6, 'change_on_settlement': -29.2, 'change_on_open_interest': 9207}
{'exchange': 'CFFEX', 'date': datetime.date(2021, 5, 21), 'symbol': 'IC2109', 'open': 6388.4, 'high': 6410.0, 'low': 6337.8, 'close': 6364.0, 'settlement': 6364.8, 'previous_settlement': 6387.6, 'volume': 11478, 'amount': 1462193.124, 'open_interest': 75895, 'change_on_close': -23.6, 'change_on_settlem

## 上期所

上期所数据文件是 xls 格式，需要使用到 xlrd 第三方库。

In [3]:
def read_shfe_data(data_file: Path) -> List[Dict[str, Any]]:
    """
    读取上海期货交易所（上期所，SHFE）的历史交易数据 (xls 文件)。
    :param data_file: Path，待读取的文件。
    :return: list，每一个项都是一个以 str 为 key 的字典。
    """
    result: List[Dict[str, Any]] = []
    
    # Read .xls files.
    workbook = xlrd.open_workbook(xls_file)
    data_sheet = workbook.sheet_by_index(0)

    xls_column_list: List[str] = [x.value for x in data_sheet.row(2)]
    mapper: Dict[str, int] = {
        'symbol': xls_column_list.index('合约'),
        'date': xls_column_list.index('日期'),
        'previous_close': xls_column_list.index('前收盘'),
        'previous_settlement': xls_column_list.index('前结算'),
        'open': xls_column_list.index('开盘价'),
        'high': xls_column_list.index('最高价'),
        'low': xls_column_list.index('最低价'),
        'close': xls_column_list.index('收盘价'),
        'settlement': xls_column_list.index('结算价'),
        'change_on_close': xls_column_list.index('涨跌1'),
        'change_on_settlement': xls_column_list.index('涨跌2'),
        'volume': xls_column_list.index('成交量'),
        'amount': xls_column_list.index('成交金额'),
        'open_interest': xls_column_list.index('持仓量'),
    }

    i: int
    last_symbol: str = ''
    for i in range(3, data_sheet.nrows - 5):
        row = data_sheet.row(i)
        if row[mapper['date']].value == 'Date':
            continue
        if row[mapper['symbol']].ctype != 0:
            last_symbol = row[mapper['symbol']].value
        product, contract = split_symbol(last_symbol)
        result.append(
            {
                'exchange': 'SHFE',
                'symbol': last_symbol,
                'product': product,
                'contract': contract,
                'date': dt.date(
                    year=int(row[mapper['date']].value[:4]),
                    month=int(row[mapper['date']].value[4:6]),
                    day=int(row[mapper['date']].value[6:8])
                ),

                'previous_close': row[mapper['previous_close']].value
                if row[mapper['settlement']].ctype != 0 else None,

                'previous_settlement': row[mapper['previous_settlement']].value
                if row[mapper['settlement']].ctype != 0 else None,

                'open': row[mapper['open']].value if row[mapper['open']].ctype != 0 else None,
                'high': row[mapper['high']].value if row[mapper['high']].ctype != 0 else None,
                'low': row[mapper['low']].value if row[mapper['low']].ctype != 0 else None,
                'close': row[mapper['close']].value if row[mapper['close']].ctype != 0 else None,
                'settlement': row[mapper['settlement']].value if row[mapper['settlement']].ctype != 0 else None,

                'change_on_close': row[mapper['change_on_close']].value
                if row[mapper['change_on_close']].ctype != 0 else None,
                'change_on_settlement': row[mapper['change_on_settlement']].value
                if row[mapper['change_on_settlement']].ctype != 0 else None,

                'volume': int(row[mapper['volume']].value)
                if row[mapper['volume']].ctype != 0 else None,

                'amount': row[mapper['amount']].value
                if row[mapper['amount']].ctype != 0 else None,

                'open_interest': int(row[mapper['open_interest']].value)
                if row[mapper['open_interest']].ctype != 0 else None,
            }
        )
    return result

# 测试
# 用前面第4轮解压的文件，选了一个离现在比较近的日期，内容比较多一些。
test_data_file: Path = PATH_FOR_RESULT.joinpath('temp4', '所内合约行情报表2020.9.xls')

for item in read_shfe_data(test_data_file):
    print(item)

NameError: name 'xlrd' is not defined